In [1]:
import sys
sys.path.insert(1,'..')
import learnHeat as lh
from importlib import reload
lh = reload(lh)
import pygsp as pg
import numpy as np
import random
import scores_table as scort
scort = reload(scort)
import matplotlib.pyplot as plt

In [ ]:
trial = np.load('trial.npy')
trial += 1
experiments = 1
N = 20
tau0 = [0.5,1,2.5,4]
Mvec = [1,3,5,7,10]
quantiles = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
alpha = 0.01
beta = 0.1
score_res = np.zeros([experiments,len(Mvec),len(quantiles),3,2])
max_iter = 50
for i in range(experiments):
    ground_laplacians = []
    learned_laplacians = []
    for M in range(Mvec):
        L = lh.heat_graph_BA(N,Mvec[M])
        ground_laplacians.append(L)
        X = lh.create_deltas(L,tau0)
        res = lh.learn_heat(X,tau0=tau0,alpha=alpha,beta=beta,max_iter=max_iter)
        L_learned = res["L"]
        L_learned[abs(L_learned<0.001)] = 0
        learned_laplacians.append(L_learned)

w
    for j in range(3):
        # do stuff
        for q in range(len(quantiles)):
            score_res[i,j,q,0,:] = scort.both_scores(lh.heat_quantile(learned_laplacians[j],quantiles[q]),ground_laplacians[j])
        for f in range(len(finesse)):
            score_res[i,j,f,1,:] = scort.both_scores(lh.heat_persistent(learned_laplacians[j],finesse[f]),ground_laplacians[j])
        for w in range(len(windows)):
            score_res[i,j,w,2,:] = scort.both_scores(lh.weight_clusters(learned_laplacians[j],windows[w]),ground_laplacians[j])
        score_res[i,j,0,3,:] = scort.both_scores(lh.heat_threshold(learned_laplacians[j],np.random.rand()),ground_laplacians[j])

np.save(f"noparams_CONV_QUANT_PERS_{trial}_experiments{experiments}_N{N}_iter{max_iter}.npy",score_res)
np.save("trial.npy",trial)

In [2]:
trial=0
np.save("trial.npy",trial)
